# Install Dependencies

In [201]:
!pip install zarr
!pip install pandas
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 2.8 MB/s eta 0:00:00


In [1]:
import s3fs
import zarr
import json
from tqdm import tqdm
import pandas as pd

# Grab Wifire Simulations Run Data 

In [2]:
endpoint = 'https://wifire-data.sdsc.edu:9000'
access_key = 'bp3d-perf'
secret_key = 'blvWRaejB1aTXPx6CqZ2GVxYbmWOSCDL'

fs = s3fs.S3FileSystem(key=access_key,
    secret=secret_key,
    client_kwargs={
        'endpoint_url': endpoint,
        'verify': False
    },
    skip_instance_cache=False
)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

name = 'quicfire.zarr'
bucket = 'burnpro3d/d'
# path = '00/16/run_001676e4-ed7a-42e4-abc5-9c6f1dc122df'
# with fs.open( bucket + '/' + path + '/' + name + '/.zattrs') as f:
#         data=json.load(f)


In [3]:
root = list(fs.ls(bucket))
simulation_paths = []

def get_paths(bucket):
    global fs
    paths = fs.ls(bucket)
    for path in tqdm(paths[:1]):
        paths = fs.ls(path)
        for path in paths:
            get_child_directories(path)

def get_child_directories(path):
    global fs
    global simulation_paths
    paths = fs.ls(path)
    for p in paths[:5]:
        if "run_" in p:
            simulation_paths.append(p)

get_paths(bucket)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.19s/it]


In [4]:
len(simulation_paths)

92

# Data Exploration

In [5]:
from pprint import pprint
def get_df(count):
    global simulation_paths
    d = {
        "run_start": [],
        "run_end": [],
        "run_binary": [],
        "sim_time":[],
        # "resolution":[]
    }
    df = pd.DataFrame(d)   
    i = 0
    for p in tqdm(simulation_paths[:count]):
        try:
            with fs.open(p + '/' + name + '/.zattrs') as f:
                data=json.load(f)
            df.loc[i] = [data.get(i) for i in d.keys()]
            i+=1
        except:
            print("FileNotFound error on path {", p,"}")
    return df

df = get_df(20)

 75%|███████████████████████████████████████████████████████████████████▌                      | 15/20 [00:05<00:01,  3.59it/s]

FileNotFound error on path { burnpro3d/d/00/20/run_0020b844-1233-4547-a9e2-5c9a11785939 }


100%|██████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:09<00:00,  2.13it/s]


In [8]:
df

,run_start,run_end,run_binary,sim_time
0,2022-04-07T15:28:35.045121-07:00,2022-04-07T20:41:23.473740-07:00,/quicfire/quicfire_Jan2022,3200.0
1,2023-07-18T14:17:12.279765-07:00,None,/quicfire/quicfire-v5.2-ucsd-rel,5983.0
2,2022-03-31T16:13:27.935085-07:00,2022-03-31T17:42:42.149814-07:00,/quicfire/quicfire_Jan2022,2100.0
3,2022-10-22T12:52:03.903821-07:00,2022-10-22T19:05:52.183442-07:00,/quicfire/quicfire-v4.0.1,18600.0
4,2023-01-09T18:33:02.835216-08:00,2023-01-09T18:41:19.260942-08:00,/quicfire/quicfire-v5.1.1,600.0
5,2023-02-01T10:33:03.706186-08:00,2023-02-01T10:46:25.591971-08:00,/quicfire/quicfire-v5.1.1,15963.0
6,NaN,NaN,NaN,100.0
7,2022-05-18T16:38:15.105115-07:00,None,/quicfire/quicfire_Jan2022,1800.0
8,2023-02-24T10:56:05.393579-08:00,2023-02-24T11:37:45.285185-08:00,/quicfire/quicfire-v5.1.1,600.0
9,2022-07-13T01:18:55.686756-07:00,2022-07-13T01:27:31.027213-07:00,/quicfire/quicfire_Jan2022,1000.0


In [10]:
# these are the top level directories, all 200
p = 'burnpro3d/d/00/0d/run_000d836a-e4d0-46c2-8ce6-021cf9ee4e9c'
with fs.open(p + '/' + name + '/.zattrs') as f:
                data=json.load(f)
pprint(data,compact=True,depth=1)
print(data["resolution"])

{'canopy_moisture': 1.5,
 'dz': None,
 'extent': [...],
 'extent_fmt': '[[x1, y1], [x2, y2]]',
 'fire_grid': None,
 'fuel': {...},
 'ignition': {...},
 'output': {...},
 'resolution': [...],
 'resolution_units': 'meters',
 'run_binary': '/quicfire/quicfire-v4.0.1',
 'run_end': '2022-10-22T19:05:52.183442-07:00',
 'run_start': '2022-10-22T12:52:03.903821-07:00',
 'seed': -1,
 'sim_time': 18600,
 'surface_moisture': 0.05,
 'threads': 1,
 'topo': None,
 'wind_direction': 245.0,
 'wind_speed': 1.0}
[1.0, 2.0, 2.0]


In [37]:
df.head(100)

,run_start,run_end,run_binary
0,2022-04-07T15:28:35.045121-07:00,2022-04-07T20:41:23.473740-07:00,/quicfire/quicfire_Jan2022
1,2023-07-18T14:17:12.279765-07:00,None,/quicfire/quicfire-v5.2-ucsd-rel
2,2022-03-31T16:13:27.935085-07:00,2022-03-31T17:42:42.149814-07:00,/quicfire/quicfire_Jan2022
3,2022-10-22T12:52:03.903821-07:00,2022-10-22T19:05:52.183442-07:00,/quicfire/quicfire-v4.0.1
4,2023-01-09T18:33:02.835216-08:00,2023-01-09T18:41:19.260942-08:00,/quicfire/quicfire-v5.1.1
...,...,...,...
91,2022-11-16T04:53:47.858409-08:00,2022-11-16T04:54:10.094242-08:00,/quicfire/quicfire-v5.0.0
92,2022-04-04T18:37:50.289153-07:00,None,/quicfire/quicfire_Jan2022
93,2022-02-28T15:49:57.081434-08:00,2022-02-28T16:01:07.748676-08:00,/ceph/QF_Versions/QF_1-6-22/source_code/compil...
94,2023-01-07T17:37:48.212893-08:00,2023-01-07T17:38:59.939612-08:00,/quicfire/quicfire-v5.1.1


In [118]:
# pandas group by version in dataframe

# caerful we have ign oor fuel object
# only look for ones that say run

In [119]:
path = '00/16/run_001676e4-ed7a-42e4-abc5-9c6f1dc122df'
# scan top directory 
# ls bucket but max step is 01

In [120]:

print("data is \n")
print(data)


data is 

{'canopy_moisture': 1, 'dimension': ['t', 'z', 'y', 'x'], 'dz': None, 'extent': [0, 600, 600, 0], 'extent_fmt': '[[x1, y1], [x2, y2]]', 'extent_origin': 'bottom left', 'fire_grid': None, 'fuel': {'density': 0.7, 'dx': 2, 'dy': 2, 'height': 1.0, 'id': None, 'xlen': 600, 'ylen': 600}, 'ignition': {'dat': None, 'id': '93826841-f991-4b94-9a5f-24b904a1a3ce', 'perc': 100}, 'output': {'emissions': False, 'energy_atmos': True, 'fire_energy': True, 'fuels_dens': True, 'fuels_moist': True, 'plume_locs': False, 'plume_traj': False, 'qu_avg_wind': False, 'qu_wind': False, 'reaction_rate': False, 'steps_fire': 1, 'steps_wind': 1, 'thermal_rad': False, 'vert_mass_burnt': False, 'wind': False}, 'resolution': [2, 2], 'resolution_units': 'meters', 'run_binary': '/quicfire/quicfire-v5.1.1', 'run_end': '2023-02-24T11:37:45.285185-08:00', 'run_max_mem_rss_bytes': '428560', 'run_start': '2023-02-24T10:56:05.393579-08:00', 'seed': -1, 'sim_time': 600, 'surface_moisture': 0.05, 'threads': 1, 'times

# other thing

In [123]:

{
ages: query_api_site()


height = {
    "01":1,
    "02":32,
    "1c":101,
    "4f":63,
}

size = {
    "01":"S",
    "02":"M",
    "1c":"XL",
    "4f":"L",
}
}

ids = ["01", "02", "1c", "4f"]
for person in ids:
    row = {
        "age":ages.get(person), 
        "height":height[person],
        "size":size[person]
    }
    df.append(row,ignore_index=True)

df.head()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (77007401.py, line 2)